<a href="https://colab.research.google.com/github/Makito042/Domain-Specific-Assistant-via-LLMs-Fine-Tuning/blob/main/train_conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Companion Plants Assistant: Fine-Tuning Gemma-2b on Colab

This notebook demonstrates how to fine-tune a Gemma-2b model to become an expert in companion planting. We will use the `unsloth` library for 2-5x faster training and 70% less memory usage, allowing this to run on a free Google Colab T4 GPU instance.

## 1. Installation

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install evaluate rouge_score sacrebleu bert_score

## 2. Load Model & Tokenizer

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

==((====))==  Unsloth 2026.2.1: Fast Gemma patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## 3. Load and Preprocess Data (Strict & Bi-directional)

In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset

# alpaca_prompt as defined previously
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # We need to add the EOS token for correct training
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# Load the JSONL file directly using datasets library
raw_dataset = load_dataset("json", data_files="/content/train_conversational.jsonl", split="train")

# Apply the formatting function to create the 'text' column
dataset = raw_dataset.map(formatting_prompts_func, batched=True)

if dataset:
    print(f"Final dataset created with {len(dataset)} examples. Includes EOS tokens.")
    print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

Final dataset created with 852 examples. Includes EOS tokens.
{'instruction': 'What plants does Alliums help grow?', 'input': '', 'output': 'Planting Alliums will benefit brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat plants does Alliums help grow?\n\n### Input:\n\n\n### Response:\nPlanting Alliums will benefit brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.<eos>'}


In [ ]:
import pandas as pd

try:
    # Read a few lines of the JSONL file to inspect its structure and column names
    df_check = pd.read_json('/content/train_conversational.jsonl', lines=True, nrows=5)
    print("Column names in train_conversational.jsonl:")
    print(df_check.columns.tolist())
    print("\nFirst 5 rows of the DataFrame:")
    display(df_check)
except FileNotFoundError:
    print("Error: /content/train_conversational.jsonl not found. Please ensure it is uploaded.")

Column names in train_conversational.jsonl:
['instruction', 'input', 'output']

First 5 rows of the DataFrame:


,instruction,input,output
0,What plants does Alliums help grow?,,"Planting Alliums will benefit brassicas, capsi..."
1,"If I plant Alliums, what other crops will bene...",,The following plants get a boost from Alliums:...
2,"If I plant Alliums, what other crops will bene...",,"Planting Alliums will benefit brassicas, capsi..."
3,I want to plant Alliums. What other plants wil...,,"The best companions for Alliums are carrots, m..."
4,What should I plant next to Alliums?,,"The best companions for Alliums are carrots, m..."


## 4. Train (High Precision)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # MAJOR CHANGE: 15 Epochs to force memorization of small dataset
        num_train_epochs = 15,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

Map (num_proc=2):   0%|          | 0/852 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 852 | Num Epochs = 15 | Total steps = 1,605
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,611,648 of 2,525,784,064 (0.78% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,5.664500
2,5.218000
3,5.473300
4,4.817000
5,4.221200
6,4.019800
7,3.242100
8,2.734700
9,2.282000
10,2.172000


TrainOutput(global_step=1605, training_loss=0.2027761056555023, metrics={'train_runtime': 3631.8299, 'train_samples_per_second': 3.519, 'train_steps_per_second': 0.442, 'total_flos': 1.2698988149071872e+16, 'train_loss': 0.2027761056555023, 'epoch': 15.0})

## 5. Quantitative Evaluation

In [ ]:
import evaluate
import torch
from tqdm import tqdm
import pandas as pd

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def calculate_perplexity(model, tokenizer, dataset):
    model.eval()
    nlls = []
    subset = dataset.select(range(min(len(dataset), 50)))
    for example in tqdm(subset, desc="Calculating Perplexity"):
        encodings = tokenizer(example["text"], return_tensors="pt")
        input_ids = encodings.input_ids.to(model.device)
        target_ids = input_ids.clone()
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            nlls.append(outputs.loss)
    return torch.exp(torch.stack(nlls).mean()).item()

def evaluate_model(model, tokenizer, dataset, num_samples=20):
    FastLanguageModel.for_inference(model)
    references = []
    predictions = []
    prompts = []
    samples = dataset.select(range(min(len(dataset), num_samples)))

    for example in tqdm(samples, desc="Generating Predictions"):
        text = example['text']
        parts = text.split("### Response:\n")
        if len(parts) > 1:
            prompt = parts[0] + "### Response:\n"
            ground_truth = parts[1].replace(tokenizer.eos_token, "").strip()
        else:
            continue

        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
        # REPETITION PENALTY ENABLED
        outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True, repetition_penalty=1.2, no_repeat_ngram_size=3)
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        if "### Response:\n" in decoded:
            pred = decoded.split("### Response:\n")[-1].strip()
        else:
            pred = decoded.strip()

        predictions.append(pred)
        references.append([ground_truth])
        prompts.append(prompt)

    if predictions:
        try:
            bleu_score = bleu.compute(predictions=predictions, references=references)
            rouge_refs = [r[0] for r in references]
            rouge_score = rouge.compute(predictions=predictions, references=rouge_refs)
            ppl = calculate_perplexity(model, tokenizer, dataset)

            print(f"\n=== Evaluation Results ===")
            print(f"BLEU: {bleu_score['bleu']:.4f}")
            print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")
            print(f"Perplexity: {ppl:.4f}")
        except Exception as e:
            print(f"Metrics calculation error: {e}")

        pd.set_option('display.max_colwidth', None)
        df_results = pd.DataFrame({
            'Input': [p.split('### Instruction:\n')[1].split('\n\n### Input:')[0][:50] for p in prompts[:5]],
            'Ground Truth': [r[0] for r in references[:5]],
            'Prediction': predictions[:5]
        })
        display(df_results)

evaluate_model(model, tokenizer, dataset)

Calculating Perplexity: 100%|██████████| 50/50 [00:11<00:00,  4.33it/s]



=== Evaluation Results ===
BLEU: 0.8181
ROUGE-L: 0.8020
Perplexity: 1.0871


,Input,Ground Truth,Prediction
0,What plants does Alliums help grow?,"Planting Alliums will benefit brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.","Planting Aliums will benefit brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes."
1,"If I plant Alliums, what other crops will benefit","The following plants get a boost from Alliums: brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.","PlantingAlliumswillbenefitbrassicas,cabbage,fruits, ?', legumes, desideri ,.turmeric."
2,"If I plant Alliums, what other crops will benefit","Planting Alliums will benefit brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.","PlantingAlliumswillbenefitbrassicas,cabbage,fruits, ?', legumes, desideri ,.turmeric."
3,I want to plant Alliums. What other plants will he,"The best companions for Alliums are carrots, marigold, marigolds, mints, pansy, peppermint, spearmint, tarragon, tomatoes.","The best companions for Allianz are carrots, marigold, marigolds, mints, pansy, peppermint, spearmint, tarragon, tomatoes."
4,What should I plant next to Alliums?,"The best companions for Alliums are carrots, marigold, marigolds, mints, pansy, peppermint, spearmint, tarragon, tomatoes.","The best companions for Alliums are carrots, marigold, marigolds, mints, pansy, peppermint, spearmint, tarragon, tomatoes."


## 6. Save Model

In [ ]:
model.save_pretrained("lora_model_final")
!zip -r lora_model_final.zip lora_model_final

  adding: lora_model_final/ (stored 0%)
  adding: lora_model_final/adapter_model.safetensors (deflated 8%)
  adding: lora_model_final/adapter_config.json (deflated 58%)
  adding: lora_model_final/README.md (deflated 65%)
